# 남은 작업 : 예외처리(try-except, 예외시마다 로그인 계정바꾸기)
# 크롤링 중 예외발생시 작업 저장, 매 요청시마다 딜레이 다르게 주기**

# 리트라이하는 로직 완성하기
# '안성기러기러기' 같이 검색결과가 없는 키워드 같은 경우..?
# 'CardiB'와 같이 검색결과가 인기게시물밖에 없는경우-> 인스타그램 자체에서 인기게시물만 반환하고 잇음
# 밴먹은 해시태그 리스트 만들어놓고 예외처리로 관리하기


In [36]:
#try except 이용한 예외처리
#sleep함수 이용해서 매 요청마다 딜레이 다르게 주기

In [37]:
from requests import get, post, Session
from urllib import parse
import json
from datetime import datetime

In [51]:
def login(id, pw, headers, session):
    resp = session.get('https://www.instagram.com/data/manifest.json', headers=headers)
    headers['x-csrftoken'] = session.cookies.get('csrftoken')
    url = 'https://i.instagram.com/api/v1/web/accounts/login/ajax/'
    params = {
        'enc_password': None,
        'username': None,
        'optIntoOneTap': 'false'
    }
    params['username'] = id #id
    params['enc_password'] = '#PWD_INSTAGRAM_BROWSER:0:{}:{}'.format(
    int(datetime.now().timestamp()), pw)   # PW
    resp = session.post(url, data=params, headers=headers) #로그인 요청
    print(resp.status_code, resp.json())
    if(resp.status_code != 200) or (not resp.json()['authenticated']):
        print(resp.text)
        print('\nlogin response error : retrying with other accounts...\n\n')
        return (headers, session, False)
    print('\nsuccessfully logged in...')
    return (headers, session, True)

In [52]:
def hot_key_instagram_top(query, headers, session, max_page=5):
    before = datetime.now()
    top_list = [[] for i in range(max_page)]
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    resp = session.get(url, headers = headers)
    if(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way\n')
        print(resp.text)
        return (False, top_list) #계정바꿔서 재시도
    
    #첫페이지 응답 정상인경우
    resource = resp.json()
    
    #top 게시물 돌기
    print('인기 게시물: 1페이지....')
    for i in resource['data']['top']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    top_list[0].append(k['media']['caption']['text'])

    #top기준 다음 페이지 관련 정보
    top_info = {'top_max_id':'', 'top_page': '', 'top_isnext': False}
    if resource['data']['top']['more_available']:
        top_info['top_max_id'], top_info['top_page'], top_info['top_isnext'] = \
        resource['data']['top']['next_max_id'], resource['data']['top']['next_page'], True
    
    #그 다음 페이지부터 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': top_info['top_max_id'], 'page': top_info['top_page'], 'surface' : 'grid', 'tab': 'top'} #tab을 top으로 변경
    
    while (top_info['top_isnext']==True and data['page']<max_page):
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        resource = resp.json()
        print('인기 게시물: {}페이지....'.format(data['page']+1))
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        top_list[data['page']].append(k['media']['caption']['text'])
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            top_info['top_isnext']=False
    
    print('인기 게시물 수집 완료!\n')
    print('총 소요시간 : ', datetime.now() - before,'\n\n')
    return (True, top_list)

In [53]:
def hot_key_instagram_recent(query, headers, session, max_page=5):
    before = datetime.now()
    recent_list = [[] for i in range(max_page)]
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    resp = session.get(url, headers = headers)
    if(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way\n')
        print(resp.text)
        return (False, recent_list) #계정바꿔서 재시도
    
    #첫페이지 응답 정상인경우
    resource = resp.json()
    
    #recent 돌기
    print('최근 게시물: 1페이지....')
    for i in resource['data']['recent']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    recent_list[0].append(k['media']['caption']['text'])
    #recent기준 다음 페이지 관련 정보
    recent_info = {'max_id':'', 'page': '', 'isnext': False}
    if resource['data']['recent']['more_available']:
        recent_info['max_id'], recent_info['page'], recent_info['isnext'] = \
        resource['data']['recent']['next_max_id'], resource['data']['recent']['next_page'], True
        
    #그 다음 페이지부터 recent 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': recent_info['max_id'], 'page': recent_info['page'], 'surface' : 'grid', 'tab': 'recent'}

    while (recent_info['isnext']==True and data['page']<max_page):
        print('최근 게시물: {}페이지....'.format(data['page']+1))
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        resource = resp.json()
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        recent_list[data['page']].append(k['media']['caption']['text'])
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            recent_info['isnext']=False
    
    print('최근 게시물 수집 완료!\n')
    print('총 소요시간 : ', datetime.now() - before,'\n\n')
    return (True, recent_list)

In [54]:
def print_text(text_list):
    for i in range(len(text_list)):
        print('\n\n\n\n\n\n################################{}페이지##########################################################################\n\n\n'.format(i+1))
        for k in text_list[i]:
            for j in k.splitlines():
                if len(j)>0:
                    print(j)

In [57]:
#MAIN---------------------------------------------------------------
keyword = '김하성'
# 초기값 선언
login_success = False
total_success = False
acc_list = [('kj10522002@korea.ac.kr','dev123dev!'),('quickpass8@gmail.com','thskrl1!'),('hotkey2','gktzl2'),
           ('lshyun0510.11@gmail.com', '*thvmxmdnpdj'), ('seungirumd+1@gmail.com', 'pinstaw24')] #lshyun0510.dev@gmail.com / *thvmxmdnpdj (성현님 예비 id/pw)
cur_idx = -1 #현재 사용하고 있는 아이디 인덱스
max_page = 3
top_list, recent_list = list(), list()
#초기값 설정 끝

In [58]:
while not total_success: #크롤링 성공까지 돌아가면서 시도
    cur_idx = (cur_idx+1)%len(acc_list) #원형 순환
    before = datetime.now()
    while not login_success: #로그인 성공까지 돌아가면서 시도
        print('Currently trying with ... :', acc_list[cur_idx][0]) #현재 로그인 시도 중인 계정
        session = Session()
        headers = dict()
        headers['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.24'
        headers['x-ig-app-id'] = '936619743392459'
        headers, session, login_success = login(acc_list[cur_idx][0], acc_list[cur_idx][1], headers, session)
        if (not login_success):
            cur_idx = (cur_idx+1)%len(acc_list) #원형 순환
    print('로그인 소요시간 : ', datetime.now() - before, '\n')
    login_success = False
    #로그인 성공인경우// 현재 cur_idx는 로그인 진입 성공한 계정 인덱스
    print('Maximum number of pages to crawl : ', max_page, '\n\n')
    total_success, top_list = hot_key_instagram_top(keyword, headers, session, max_page)
    #or
    total_success, recent_list = hot_key_instagram_recent(keyword, headers, session, max_page)

total_success = False

Currently trying with ... : kj10522002@korea.ac.kr
403 {'message': 'Sorry, your password was incorrect. Please double-check your password.', 'status': 'fail'}
{"message":"Sorry, your password was incorrect. Please double-check your password.","status":"fail"}

login response error : retrying with other accounts...


Currently trying with ... : quickpass8@gmail.com
200 {'user': True, 'userId': '56084381378', 'authenticated': True, 'oneTapPrompt': True, 'status': 'ok'}

successfully logged in...
로그인 소요시간 :  0:00:05.730693 

Maximum number of pages to crawl :  3 


인기 게시물: 1페이지....
인기 게시물: 2페이지....
인기 게시물: 3페이지....
인기 게시물 수집 완료!

총 소요시간 :  0:00:07.708823 


최근 게시물: 1페이지....
최근 게시물: 2페이지....
최근 게시물: 3페이지....
최근 게시물 수집 완료!

총 소요시간 :  0:00:06.195333 




In [19]:
top_list, recent_list

([['~7.12\n차일피일 미루다가 올립니다ㅠ\n방학은 잘들 보내고 계신가요? 저는 할일이 여러 개 있어서 조금 정신없이 보내고 있어요. 공부한 것도 도저히 업로드할 퀄리티가 아닌 것 같았지만 다시 꾸준히 올려보겠습니다. 피드 잘 안올리는 와중에도 조금씩이지만 팔로워 수가 늘어서 신기하고 또 감사하네요. 앞으로도 자극 받을 수 있는 공스타 운영하도록 노력해볼게요. \n잘 정리된 형태가 아니더라도 꾸준히 인증하는걸 목표로 할 생각입니다. \n\n<List> \n- 통계, ai/ml notion에 요약\n- h모빌리티 요약\n- 논문 진행사항 체크\n\n#공스타 #공부인증 #공대생 #공대4학년 #공스타맞팔 #공스타소통 #데이터사이언스 #통계 #머신러닝 #딥러닝 #h모빌리티클래스 #할일이산더미 #여름방학',
   '스스로 사고할 수 있는 인공지능의 등장! 사람보다 더 똑똑한 ‘#초거대ai ’란 무엇일까?\n\n최근 10년 사이 AI의 활용도가 급증하고 있다. \n특히 인공지능의 능력이 초거대로 성장하며 사람보다 더 똑똑한 AI가 나타나고 있다. 바로 대용량 데이터를 스스로 학습해 종합적 추론이 가능한 ‘초거대 AI’이다. \n\n팬데믹 이후 디지털 혁신이 가속화되며 초거대 AI의 활용 범위 또한 확대되고 있다. 거동이 불편한 사람도 웨어러블 로봇으로 일상생활이 가능하며, 영유아부터 노인까지 AI를 통해 맞춤형 돌봄 서비스가 가능하다. \n\n더불어 인간과 협업해 창작예술에 뛰어드는 사례도 늘고 있다는데! 이처럼 AI의 발전으로 생활 속 편리함은 더해지고 있지만, 동시에 많은 우려들이 쏟아지고 있다. 예를 들면 ‘AI가 인간을 해치지는 않을까?’ ‘일자리를 위협하는 건 아니야?’와 같은 의구심이다.\n\n잘 모르기에 더 두렵고 궁금한 AI의 세계! #ebs비즈니스리뷰 #배순민 소장님의 지금 당신 옆에 AI가 있다’ 편 #많관부 \U0001faf6🏻\n\n1부. 초거대 AI가 온다\n2부. 어느 날 AI가 내게 말을 걸어왔다\n3부. AI는 왜 그림을 그릴까 \n4부

In [98]:
print('인기게시물 출력 : ')
print_text(top_list)

인기게시물 출력 : 






################################1페이지##########################################################################



둘 다 첫 고연전🐯🐯
.
.
.
#고연전 #연고전 #고려대 #서울 #대학생 #l4l #f4f #fff #like4likes #팔로우 #맞팔 #좋반 #좋아요반사 #고려대학교 #연세대학교 #고양
즐거운 고연전 날에🎶
#고려대 #고연전 #압승
퍼스널 컬러❤️ #고연전 #고려대학교
당신에게 세계를,
2021. 
현영
#2022정기고연전 #농구
20221028 / 고양체육관
#정기고연전 #정기연고전 
#고연전 #연고전
#고려대학교응원단 #고려대학교
#잠실야구장
#디카츄_고려대학교응원단 
#디카츄_마음의고향 
#디카츄_Sports
#디카츄의사진창고 #디카츄
#출근전업로드
A Night to Remember 🔴⚫️⚪️
.
.
.
.
.
 #KoyeonClassic #Descente #고연전 #정기전 #빙구 #아이스하키 #포카리스웨트 #데쌍트 #62
#2022정기고연전 #야구
20221028 / 잠실야구장
#정기고연전 #정기연고전 
#고연전 #연고전
#고려대학교응원단 #고려대학교
#잠실야구장
#디카츄_고려대학교응원단 
#디카츄_마음의고향 
#디카츄_Sports
#디카츄의사진창고 #디카츄
너에겐 가장 예쁜 모습만 보여주고 싶어서. 
2021.
#2022정기고연전 #농구
20221028 / 고양체육관
#정기고연전 #정기연고전 
#고연전 #연고전
#고려대학교응원단 #고려대학교
#잠실야구장
#디카츄_고려대학교응원단 
#디카츄_마음의고향 
#디카츄_Sports
#디카츄의사진창고 #디카츄






################################2페이지##########################################################################



인류애 바사삭했다가 풀충👏👏👏👏
#좋아요 #팔로우 격하

In [99]:
print('최근게시물 출력 : ')
print_text(recent_list)

최근게시물 출력 : 






################################1페이지##########################################################################



고연전🐯너무나 소중한 추억
#고연전 #고려대학교 #승리
You know you had a good time when 민족의 아리아 is ringing in your ears when you try to sleep 
#고연전 #고연전🐯
진지하게 ,, 연대 응원이 .. 재밌어요 .. ?
 - 고연전 2일차
안되겠다 너무 재밌따 졸업해도 가야겠뎌🫢🐯❤️‍🔥❤️‍🔥
#고연전 #필승전승압승 #이겨버림
빨리
오라
.
.
.
#주말
#정기전
#고연전
#연고전
#럭비
#2022정기고연전 #주경기장 #럭비 #축구
20221029 / 고양종합운동장 
#정기고연전 #정기연고전 
#고연전 #연고전
#고려대학교응원단 #고려대학교
#잠실야구장
#디카츄_고려대학교응원단 
#디카츄_마음의고향 
#디카츄_Sports
#디카츄의사진창고 #디카츄
주말 강원도 1박2일 투어
버치힐 그리고 알펜시아
너무 좋은 구장에서 좋은 분들과 고연전 대결
#골프 #골프스타그램 #벤처기업 #청년사업가 #이노비즈 #메인비즈 #경치가예술 #버치힐cc #알펜시아cc #날씨좋다 #가을골프 #golf #사업가 #싱글치고싶다 #무기명 #회원제 #고연전 #1박2일골프
🐯막학기 고연전은 행복하고 잊지 못할 추억이 될 것임❤️
#고연전 #고려대 #고려대학교 #연세대 #연세대학교
#고연전 때 발급받은 버미 #nft 
황금호랑이도 있다고하는데 괜히 가지고싶다
'1인1개 한정'이라 희소성때문에 그렇겠지
멋있어! 멋있는 학생들이다
#일상 #좋아요 #좋반 #오오티디 #여행 #유머 #꿀잼 #정보 #팔로우반사 #공감 #강아지 #남자 #여자 #고양이 #휴식 #휴가 #음식 #고대 #안암 #에타 #인류애 #충전 #청소 #연고전 #고연전 #쓰레기 #호잇쨈
늦은 고연전 인증
Photo by @jjun_howoo 
#